# Unsupervised user explicit rating based recommendation system

## Import Libraries

In [1]:
# Standard library imports
import os # allows access to OS-dependent functionalities
import re #  regular expression matching operations similar to those found in Perl
import sys # to manipulate different parts of the Python runtime environment
import warnings # is used to display the message Warning
import pickle # serializing and deserializing a Python object structure.

# Third party libraries
from fuzzywuzzy import fuzz # used for string matching

import numpy as np # functions for working in domain of linear algebra, fourier transform, matrices and arrays
import pandas as pd # data analysis and manipulation tool
import joblib # set of tools to provide lightweight pipelining in Python

# deal with sparse data libraries
from scipy.sparse import csr_matrix # Returns a copy of column i of the matrix, as a (m x 1) CSR matrix (column vector).

# visualization
import matplotlib.pyplot as plt # collection of functions that make matplotlib work like MATLAB.

## scikit Preprocessing data libraries
# package provides several common utility functions and 
# transformer classes to change raw feature vectors into 
# a representation that is more suitable for the downstream estimators.
from sklearn.preprocessing import MinMaxScaler # Transform features by scaling each feature to a given range.

## scikit Feature Extraction libraries
# Can be used to extract features in a format supported 
# by machine learning algorithms from datasets consisting 
# of formats such as text and image.
from sklearn.feature_extraction.text import TfidfVectorizer # Convert a collection of raw documents to a matrix of TF-IDF features
from sklearn.feature_extraction.text import CountVectorizer # Convert a collection of text documents to a matrix of token counts.

## scikit Pairwise metrics libraries
#implements utilities to evaluate pairwise distances or affinity of sets of samples.
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.metrics.pairwise import linear_kernel 

## scikit Cross validation iterators libraries
from sklearn.model_selection import GridSearchCV

# Unsupervised learner for implementing neighbor searches.
from sklearn.neighbors import NearestNeighbors

# setting display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Utils libraries
from utils import utils
from utils import cleaning
from utils import recommend
from utils import testing
from utils import training

#Preparing folder variables
os.chdir(os.path.dirname(sys.path[0])) # This command makes the notebook the main path and can work in cascade.
main_folder = sys.path[0]
data_folder = (main_folder + "\data")
saved_models_folder = (data_folder + "\saved_models")
raw_data = (data_folder + "\_raw")
processed_data = (data_folder + "\processed")
content_based_supervised_data = (data_folder + "\processed\content_based_supervised")


c:\Users\christiandda\miniconda3\envs\stlit\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Cleaning and preparing the data

### Importing data

In [2]:
anime = cleaning.anime()
rating = cleaning.rating()

### Merging dataframes

In [3]:
anime_cleaned = cleaning.clean_anime_df(anime)

In [4]:
merged_df = cleaning.merging(anime_cleaned)

merged_df.head(5)

,anime_id,name,genre,type,episodes,rating,members,user_id,user_rating
0,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0,99,5
1,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0,152,10
2,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0,244,10
3,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0,271,10
4,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0,278,-1


In [5]:
merged_df.shape

(7813613, 9)

### Handling NaN values

In [6]:
features = cleaning.features_user_based_unsupervised(merged_df)

features.head(5)

,anime_id,name,genre,type,episodes,rating,members,user_id,user_rating
2,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0,244,10.0
3,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0,271,10.0
7,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0,462,8.0
11,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0,598,10.0
14,32281,kimi no na wa,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630.0,784,9.0


In [7]:
features.shape

(3179694, 9)

### Pivoting

In [8]:
pivot_df = cleaning.create_pivot_table_unsupervised(features)
pivot_df.head(2)

user_id               5      7      17     38     43     46     123    129    \
name                                                                           
                        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 hack  g u  returner    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

user_id               139    160    210    226    233    235    244    248    \
name                                                                           
                        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 hack  g u  returner    0.0    8.0    0.0    0.0    0.0    0.0    0.0    0.0   

user_id               250    256    261    271    282    288    294    301    \
name                                                                           
                        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 hack  g u  returner    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

user_id               308    317    321    326    341    352    385    392    \
name                                                                           
                        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 hack  g u  returner    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

user_id               395    418    428    436    438    439    446    455    \
name                                                                           
                        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 hack  g u  returner    0.0    0.0    0.0    0.0    0.0    0.0    5.0    0.0   

user_id               460    462    478    492    497    500    511    529    \
name                                                                           
                        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 hack  g u  returner    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

user_id               530    531    540    541    547    551    553    565    \
name                                                                           
                        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 hack  g u  returner    0.0    0.0    0.0    0.0    0.0    0.0    8.0    0.0   

user_id               567    572    573    578    585    593    598    611    \
name                                                                           
                        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 hack  g u  returner    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

user_id               634    637    656    659    660    661    662    670    \
name                                                                           
                        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 hack  g u  returner    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

user_id               680    687    692    694    702    721    735    741    \
name                                                                           
                        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 hack  g u  returner    0.0    0.0    0.0    0.0    0.0    0.0    0.0    7.0   

user_id               746    750    762    768    771    774    783    784    \
name                                                                           
                        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 hack  g u  returner    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

user_id               786    795    799    806    813    814    819    820    \
name                                                                           
                        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 hack  g u  returner    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

user_id               847    861    890    894    916    920    926    936    \
name                                                                

In [9]:
pivot_df.shape

(9784, 8713)

## Recommendation building phase

### Cosine Similarity using KNN

In [10]:
model_knn = training.matrix_creation_and_training(pivot_df)

In [11]:
model_knn

NearestNeighbors(algorithm='brute', metric='euclidean', n_neighbors=2)

# Getting Recommendations

In [12]:
# We can get the recommendation as a dataframe
# We select the name of the anime we want to find similitudes and Then the number of suggestions we have(we might get less if there not so many o none if there is no matches)
# Then the genre we want (or write "All")
# Then the type we want (or write "All")

from utils import recommend
recommend.create_df(recommend.unsupervised_user_based_recommender("Monster",30),"All","All")

These are the recommendations for similar animes to monster 



,name,genre,type,episodes,rating
,ginga eiyuu densetsu,"Drama, Military, Sci-Fi, Space",OVA,110.0,9.11
,romeo no aoi sora,"Adventure, Drama, Historical, Slice of Life",TV,33.0,8.47
,gyakkyou burai kaiji hakairoku hen,"Game, Psychological, Seinen, Thriller",TV,26.0,8.29
,ashita no joe,"Drama, Sports",TV,79.0,8.22
,ginga eiyuu densetsu arata naru tatakai no overture,"Action, Drama, Military, Sci-Fi, Space",Movie,1.0,8.21
,ginga eiyuu densetsu gaiden senoku no hoshi senoku no hikari,"Action, Military, Sci-Fi, Space",OVA,24.0,8.20
,ginga eiyuu densetsu gaiden rasen meikyuu,"Drama, Military, Sci-Fi, Space",OVA,28.0,8.10
,ginga eiyuu densetsu waga yuku wa hoshi no taikai,"Action, Historical, Military, Sci-Fi, Space",Movie,1.0,8.09
,master keaton ova,"Adventure, Mystery",OVA,15.0,7.91
,oniisama e,"Drama, School, Shoujo, Shoujo Ai",TV,39.0,7.88


In [15]:
# We can get the recommendation as a dictionary
# We select the name of the anime we want to find similitudes and Then the number of suggestions we have(we might get less if there not so many o none if there is no matches)
# Then the genre we want (or write "All")
# Then the type we want (or write "All")
from utils import recommend
recommend.create_dict(recommend.unsupervised_user_based_recommender("Monster",30),"All","All")

These are the recommendations for similar animes to monster 



[{'name': 'ginga eiyuu densetsu',
  'genre': 'Drama, Military, Sci-Fi, Space',
  'type': 'OVA',
  'episodes': 110.0,
  'rating': 9.11},
 {'name': 'romeo no aoi sora',
  'genre': 'Adventure, Drama, Historical, Slice of Life',
  'type': 'TV',
  'episodes': 33.0,
  'rating': 8.47},
 {'name': 'gyakkyou burai kaiji  hakairoku hen',
  'genre': 'Game, Psychological, Seinen, Thriller',
  'type': 'TV',
  'episodes': 26.0,
  'rating': 8.29},
 {'name': 'ashita no joe',
  'genre': 'Drama, Sports',
  'type': 'TV',
  'episodes': 79.0,
  'rating': 8.22},
 {'name': 'ginga eiyuu densetsu  arata naru tatakai no overture',
  'genre': 'Action, Drama, Military, Sci-Fi, Space',
  'type': 'Movie',
  'episodes': 1.0,
  'rating': 8.21},
 {'name': 'ginga eiyuu densetsu gaiden  senoku no hoshi  senoku no hikari',
  'genre': 'Action, Military, Sci-Fi, Space',
  'type': 'OVA',
  'episodes': 24.0,
  'rating': 8.2},
 {'name': 'ginga eiyuu densetsu gaiden  rasen meikyuu',
  'genre': 'Drama, Military, Sci-Fi, Space',
